In [ ]:
import os                          # for operating system work
from dotenv import load_dotenv     # to load environment variables 
from langchain_groq import ChatGroq # for the chat module

load_dotenv() # loading all the environment variables
groq_api_key = os.getenv("GROQ_API_KEY") # get the groq api key

# Create Model from Groq, using the API Key and Model name
model = ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model





ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001C044337250>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001C044337C50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi, I am Dinesh, I work at Accenture as an Associate Director")])

AIMessage(content="Hi Dinesh, it's nice to meet you! \n\nThat's great you're an Associate Director at Accenture. What kind of work do you do there?  \n\nI'm eager to learn more about your role and experience.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 55, 'prompt_tokens': 25, 'total_tokens': 80, 'completion_time': 0.1, 'prompt_time': 0.002310808, 'queue_time': 0.11594836, 'total_time': 0.102310808}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--d3c569e4-144d-4095-92d9-642faebfee51-0', usage_metadata={'input_tokens': 25, 'output_tokens': 55, 'total_tokens': 80})

In [22]:
from langchain_core.messages import AIMessage

model.invoke(
    [
        HumanMessage(content="Hi, I am Dinesh, I work at Accenture as an Associate Director"),
        AIMessage(content="Hi Dinesh, it's nice to meet you! \n\nThat's great you're an Associate Director at Accenture. What kind of work do you do there?  \n\nI'm eager to learn more about your role and experience.\n"),
        HumanMessage(content="Hey whats my name and what do I do")
    ]

)

AIMessage(content="You are Dinesh, and you are an Associate Director at Accenture.  \n\nIs there anything else you'd like to tell me about your work or anything specific you'd like to discuss? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 97, 'total_tokens': 143, 'completion_time': 0.083636364, 'prompt_time': 0.005231385, 'queue_time': 0.091177177, 'total_time': 0.088867749}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--6cec5ddb-8aa0-4542-832d-b88c9d4e0656-0', usage_metadata={'input_tokens': 97, 'output_tokens': 46, 'total_tokens': 143})

In [23]:
### Message History
#This imports the ChatMessageHistory class, which is a 
# concrete implementation of BaseChatMessageHistory. It's used to store chat messages.
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
# This is a crucial class that wraps a runnable (like our ChatGroq model) and 
# imbues it with the ability to manage conversation history.
from langchain_core.runnables.history import RunnableWithMessageHistory

#This line initializes an empty dictionary named store
store={}
#This defines a function named get_session_history that takes a sessionid (a string) as 
# input and is type-hinted to return an object conforming to BaseChatMessageHistory.
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

#This is where the magic happens for incorporating message history.
#RunnableWithMessageHistory is instantiated with two arguments:
# model: This is the ChatGroq model instance we created earlier. 
# This is the "runnable" that will be enhanced with history capabilities.
# later when the invoke method is called on with_message_history, it will use the 
# stored model and get_session_history to call get_session_history

with_message_history=RunnableWithMessageHistory(model,get_session_history)

In [24]:
#This dictionary defines the configuration for the invocation.
# "configurable": This key is used to pass configuration options to 
# runnables that support them.

config={"configurable":{"session_id":"chat1"}}

In [25]:
#This line invokes the with_message_history runnable.
response = with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config #This passes the config dictionary we defined, which includes the session_id
)

In [14]:
response.content

"Hi Dinesh, it's great to meet you!  Accenture is a huge company, so I'm curious - what area of work do you specialize in?  \n\nBeing an Associate Director is a leadership position, so what are some of the biggest challenges and rewards you've found in that role?\n"

In [26]:
## change the config, it should not remember
config1={"configurable":{"session_id":"chat2"}}
with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=config1 #This passes the config dictionary we defined, which includes the session_id
)


AIMessage(content="As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to remember it! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 14, 'total_tokens': 53, 'completion_time': 0.070909091, 'prompt_time': 0.002788692, 'queue_time': 0.18181709399999998, 'total_time': 0.073697783}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--20ccc02c-53b6-4d50-827d-8c0183d40b87-0', usage_metadata={'input_tokens': 14, 'output_tokens': 39, 'total_tokens': 53})

In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
#ChatPromptTemplate.from_messages is a class method (a method called directly on the class, 
# not an instance) that allows you to construct a ChatPromptTemplate from a list of "message templates.
# " Each element in the list represents a part of the conversation, specifying who is speaking (role)
#  and what their message is (content).It's designed specifically for chat models (like your ChatGroq model)
#  because chat models understand conversational turns (system messages, human messages, AI messages, etc.)
prompt=ChatPromptTemplate.from_messages (
    [
        ("system","You are a help assitant. Answer all the questions to the best of your abaility"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [28]:
chain.invoke({"messages":[HumanMessage(content="My name is Dinesh")]})

AIMessage(content="Hello Dinesh!\n\nIt's nice to meet you. I'm happy to help with any questions you have.  Just ask away! 😊 \n\nWhat can I do for you today? \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 46, 'prompt_tokens': 32, 'total_tokens': 78, 'completion_time': 0.083636364, 'prompt_time': 0.002301304, 'queue_time': 0.090613052, 'total_time': 0.085937668}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--005a55ff-7da2-47bf-9bbb-922078c1acbf-0', usage_metadata={'input_tokens': 32, 'output_tokens': 46, 'total_tokens': 78})

In [30]:
## Create a new config called chat3
config1={"configurable":{"session_id":"chat3"}}
with_message_history=RunnableWithMessageHistory(model,get_session_history)

response = with_message_history.invoke(
    [HumanMessage(content="My name is Dinesh")],
    config=config1 #This passes the config dictionary we defined, which includes the session_id
)

response.content


"Hi Dinesh, it's nice to meet you!\n\nIs there anything I can help you with today? 😊\n"

In [ ]:
# Add more complexity, change the prompt to take a variable language 
prompt=ChatPromptTemplate.from_messages (
    [
        ("system","You are a help assitant. Answer all the questions to the best of your abaility in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
# I have to rerun the prompt here because the defination is stored and its not a link
chain=prompt|model

In [ ]:

#ChatPromptTemplate is designed to take a dictionary as its input if it contains multiple variables.
# It looks for keys in this input dictionary that match the placeholders

response = chain.invoke(
    {"messages":[HumanMessage(content="My name is Dinesh")],
     "language":"hindi"
     }
)
response.content

'नमस्ते, Dinesh! 😊  मैं आपकी मदद करने के लिए यहाँ हूँ।  आपके कोई सवाल है?  \n\n'

In [37]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
    )


In [39]:
## Create a new config called chat4
config1={"configurable":{"session_id":"chat4"}}
response4=with_message_history.invoke(
    {"messages":[HumanMessage(content="My name is Dinesh")],
     "language":"Hindi"},
    config=config1
)
response4.content

'नमस्ते दिनेश! 😊 \n\nमुझे आपकी मदद करने में खुशी होगी। आपके कोई सवाल हैं? \n\nआप मुझसे कुछ भी पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा आपको सही जवाब देना। \n\n'

In [ ]:
from langchain_core.messages import SystemMessage, trim_messages
#trim_messages: This is a crucial new component. It's a runnable factory 
# (meaning, it's a function that returns a Runnable instance) designed to intelligently shorten
#  a list of chat messages based on a token limit. This is vital for managing LLM context 
# windows and controlling costs

#trim_messages: This is a crucial new component. It's a runnable factory (meaning, 
# it's a function that returns a Runnable instance) designed to intelligently shorten a list of chat 
# messages based on a token limit. This is vital for managing LLM context windows and controlling costs
trimmer=trim_messages(
    max_tokens = 60, #This sets the maximum number of tokens the trimmed message history should occupy. 
    strategy = "last", #This dictates how messages are removed if max_tokens is exceeded.
    token_counter=model, #This is very important. trim_messages needs a way to count tokens accurately. 
                         #By passing your model (the ChatGroq instance), you're providing a 
                         # token counter that understands how Groq counts tokens, ensuring the max_tokens 
                         # limit is respected precisely for your chosen LLM.
    include_system=True, #the system message (if present) will always be included
    allow_partial=False, #If True, it might allow a message to be partially included 
                         #if splitting it fits the token limit.
    start_on="human" #means it will ensure that the first message
                     # kept in the history, after system messages, is a HumanMessag

)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm Diensh"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like Kheer Mohan Bengali Sweet"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

#initially max_tokens were set to 70 and it returned all value, when changed to 45 it trimmed messages


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm Diensh", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like Kheer Mohan Bengali Sweet', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [ ]:
# itemgetter: This is a utility function from Python's operator module.
# It's often used in LangChain's LCEL for extracting specific items (keys) from an input dictionary.
from operator import itemgetter
#This is another powerful LCEL component. It literally "passes through" its input as its output.
# It's often used with .assign() to inject or modify keys in the input dictionary without changing 
# the original input.
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    |prompt
    |model
)
response=chain.invoke(
    {
        "messages": messages + [HumanMessage(content="Which Bengali Sweet, I like?")],
        "language":"English"
    }
)

response.content

"You said you like Kheer Mohan! 😊  \n\nIs there anything else you'd like to chat about?  Perhaps other types of sweets, or something completely different?  \n"